# Libraries  

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv

# Getting the source code webpage

In [2]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text 
soup = BeautifulSoup(source, 'lxml')

# Creating a csv File and adding the columns name to it

In [3]:
csv_file = open('toronto_postal_codes.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['Postcode', 'Borough', 'Neighbourhood'])

32

# Scrapping the webpage to only get the table

In [5]:
table = soup.find('table', class_ = 'wikitable sortable') # Gets the table from the webpage
rows = table.find_all('tr') # Gets the table rows

postcodes = [] # Initializes the raw postcodes list
boroughs = [] # Initializes the raw boroughs list
neighbourhoods = [] # Initializes the raw neighbourhoods list

for row in rows:    
    columns = row.find_all('td')
    try :
        if columns[1].text != 'Not assigned':  # To skip if the borough name is 'Not Assigned'
            
            postcode = columns[0].text
            postcodes.append(postcode)
            
            borough = columns[1].text
            boroughs.append(borough)
            
            neighbourhood = columns[2].text.split('\n')[0] # Removing the newline character at the end     
            
            if neighbourhood == 'Not assigned': # Assigning the same name to neighbourhood if it is 'Not Assigned'
                neighbourhood = borough            
                
            neighbourhoods.append(neighbourhood)
             
    except Exception as e : # To skip the first row which contains column names
        pass 
    
postcode_explored = [] # Initializing the list of explored postcodes
for index_i, postcode_i in enumerate(postcodes) :   
    if postcode_i not in postcode_explored :
        nbds = neighbourhoods[index_i]
        for index_f, postcode_f in enumerate(postcodes) :
            if postcode_i == postcode_f and index_i != index_f:
                nbds = nbds + ', ' + neighbourhoods[index_f] # Concatenating the neighbourhood names
        csv_writer.writerow([postcode_i, boroughs[index_i], nbds]) # Writing the rows in the csv file
        postcode_explored.append(postcode_i)

# Close the csv file

In [6]:
csv_file.close()

# Creating the dataframe

In [8]:
df = pd.read_csv('toronto_postal_codes.csv')

# Checking the dataframe

In [9]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A\r\n,Not assigned\r\n,Not assigned\r\n
1,M2A\r\n,Not assigned\r\n,Not assigned\r\n
2,M3A\r\n,North York\r\n,Parkwoods
3,M4A\r\n,North York\r\n,Victoria Village
4,M5A\r\n,Downtown Toronto\r\n,"Regent Park, Harbourfront"


Notice that there are "\r\n" in all the dataframe, let's remove them

# Removing the "\r\n"

In [12]:
df['Postcode'] = df['Postcode'].replace('\r\n','', regex=True)
df['Borough'] = df['Borough'].replace('\r\n','', regex=True)
df['Neighbourhood'] = df['Neighbourhood'].replace('\r\n','', regex=True)

# Result:

In [13]:
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


# Creating a list of postal codes

In [14]:
postal_codes = df['Postcode'].values

# GeoCoder API credentials

In [19]:
API_KEY=''#put your API credentials here, i removed mine after compiling

# Using the OpenCage GeoCoder API

In [16]:
import json

latitudes = [] # Initializing the latitude array
longitudes = [] # Initializing the longitude array

for postal_code in postal_codes : 
    place_name = postal_code + " Toronto" # Formats the place name
    url = 'https://api.opencagedata.com/geocode/v1/json?q={}&key={}'.format(place_name, API_KEY) # Gets the proper url to make the API call
    obj = json.loads(requests.get(url).text) # Loads the JSON file in the form of a python dictionary
    
    results = obj['results'] # Extracts the results information out of the JSON file
    lat = results[0]['geometry']['lat'] # Extracts the latitude value
    lng = results[0]['geometry']['lng'] # Extracts the longitude value
    
    latitudes.append(lat) # Appending to the list of latitudes
    longitudes.append(lng) # Appending to the list of longitudes

# Adding the latitude and longitude to df

In [17]:
df['Latitude'] = latitudes
df['Longitude'] = longitudes

In [18]:
df.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1A,Not assigned,Not assigned,-34.889942,-56.079098
1,M2A,Not assigned,Not assigned,-34.889942,-56.079098
2,M3A,North York,Parkwoods,-34.889942,-56.079098
3,M4A,North York,Victoria Village,43.727600,-79.314800
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600


# Libraries for machine learning/data visualization

In [20]:
# Downloading folium, if not installed
#!conda install -c conda-forge folium=0.5.0 --yes
import folium # Map plotting library
import numpy as np
from pandas.io.json import json_normalize # Tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# Import k-means from clustering stage
from sklearn.cluster import KMeans

# FOURSQUARE API CREDENTIALS

In [40]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20190425' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


# Toronto Map using folium

In [23]:
# Toronto latitude and longitude using Google search
tor_lat = 43.6532
tor_lng = -79.3832

# Creates map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[tor_lat, tor_lng], zoom_start=10)

# Add markers to map
for lat, lng, borough, neighbourhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

# Defining own function to get the name of the category

In [24]:
# Gets the name of the category

def get_category_type(row):
    try:
        categories_list = row['Category']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Using the FourSquare API on all neighbourhoods

In [25]:
explore_df_list = []

for i, nbd_name in enumerate(df['Neighbourhood']):  
    
    try :         
        ### Getting the data of neighbourhood
        nbd_name = df.loc[i, 'Neighbourhood']
        nbd_lat = df.loc[i, 'Latitude']
        nbd_lng = df.loc[i, 'Longitude']

        radius = 500 # Setting the radius as 500 metres
        LIMIT = 100 # Getting the top 100 venues

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={} \
        &client_secret={}&ll={},{}&v={}&radius={}&limit={}'\
        .format(CLIENT_ID, CLIENT_SECRET, nbd_lat, nbd_lng, VERSION, radius, LIMIT)

        results = json.loads(requests.get(url).text)
        results = results['response']['groups'][0]['items']

        nearby = json_normalize(results) # Flattens JSON

        # Filtering the columns
        filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
        nearby = nearby.loc[:, filtered_columns]

        # Renaming the columns
        columns = ['Name', 'Category', 'Latitude', 'Longitude']
        nearby.columns = columns

        # Gets the categories
        nearby['Category'] = nearby.apply(get_category_type, axis=1)

        # Gets the data required
        for i, name in enumerate(nearby['Name']):
            explore_df_list.append([nbd_name, nbd_lat, nbd_lng] + nearby.loc[i, :].values.tolist())
    
    except Exception as e:
        pass

<ipython-input-25-4dd739e598ef>:21: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby = json_normalize(results) # Flattens JSON


# Creating a dataframe for performing clustering operations

In [26]:
explore_df = pd.DataFrame([item for item in explore_df_list])
explore_df.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue Name', 'Venue Category', 'Venue Latitude', 'Venue Longitude']
explore_df.head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue Name,Venue Category,Venue Latitude,Venue Longitude
0,Not assigned,-34.889942,-56.079098,El Rey del Chivito,Spanish Restaurant,-34.893113,-56.077577
1,Not assigned,-34.889942,-56.079098,Plaza Suiza,Plaza,-34.892200,-56.078327
2,Not assigned,-34.889942,-56.079098,Magma,Women's Store,-34.892971,-56.077550
3,Not assigned,-34.889942,-56.079098,Porto Vanila,Fast Food Restaurant,-34.893044,-56.077533
4,Not assigned,-34.889942,-56.079098,Homero Bar,Restaurant,-34.892967,-56.079695


# Total number of unique categories

In [27]:
len(explore_df['Venue Category'].unique())

256

# Performing one-hot encoding to analyze neighbourhoods

In [28]:
# One hot encoding
toronto_onehot = pd.get_dummies(explore_df[['Venue Category']], prefix="", prefix_sep="")

# Add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = explore_df['Neighbourhood'] 

# Move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio
0,Not assigned,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Not assigned,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Not assigned,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,Not assigned,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Not assigned,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


# Grouping venues by neighbourhoods

In [29]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head()

,Neighbourhood,Accessories Store,Afghan Restaurant,Airport,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,...,Video Store,Vietnamese Restaurant,Warehouse Store,Whisky Bar,Wine Bar,Wine Shop,Wings Joint,Women's Store,Xinjiang Restaurant,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.047619,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Creating a function to return the most common venues

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

# Creating a new dataframe to get the top 10 venues

In [31]:

num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
neighbourhoods_venues_sorted = pd.DataFrame(columns=columns)
neighbourhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighbourhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighbourhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Breakfast Spot,Badminton Court,Shanghai Restaurant,Skating Rink,Newsagent,Flea Market,Fish Market,Fish & Chips Shop,Ethiopian Restaurant
1,"Alderwood, Long Branch",Coffee Shop,Sandwich Place,Pizza Place,Pharmacy,Pub,Convenience Store,Gym,Farmers Market,Escape Room,Ethiopian Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Deli / Bodega,Pizza Place,Fried Chicken Joint,Mediterranean Restaurant,Coffee Shop,Middle Eastern Restaurant,Diner,Discount Store,Food Stand,Food Court
3,Bayview Village,Gas Station,Trail,Flower Shop,Park,Field,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Yoga Studio
4,"Bedford Park, Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Comfort Food Restaurant,Greek Restaurant,Butcher,Pharmacy,Liquor Store,Sushi Restaurant,Restaurant


# K-Means Clustering

In [32]:
# Set number of clusters
kclusters = 5
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(toronto_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# Add clustering labels
neighbourhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# Creating a grand dataframe containing the necessary data

In [33]:
toronto_merged = df
toronto_merged = toronto_merged.join(neighbourhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.dropna(inplace = True)
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M1A,Not assigned,Not assigned,-34.889942,-56.079098,1,Coffee Shop,Clothing Store,Plaza,Restaurant,Fast Food Restaurant,Hotel,Seafood Restaurant,Cosmetics Shop,Women's Store,Breakfast Spot
1,M2A,Not assigned,Not assigned,-34.889942,-56.079098,1,Coffee Shop,Clothing Store,Plaza,Restaurant,Fast Food Restaurant,Hotel,Seafood Restaurant,Cosmetics Shop,Women's Store,Breakfast Spot
2,M3A,North York,Parkwoods,-34.889942,-56.079098,1,Spanish Restaurant,Fast Food Restaurant,Women's Store,Plaza,Supermarket,Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
3,M4A,North York,Victoria Village,43.727600,-79.314800,1,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,French Restaurant,Pizza Place,Park,Fast Food Restaurant,Ethiopian Restaurant,Event Space
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655500,-79.362600,1,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Food Truck,Spa,Dance Studio,Event Space,Beer Store,Electronics Store


# Visualizing the clusters

In [34]:
# Create map
map_clusters = folium.Map(location=[tor_lat, tor_lng], zoom_start=11)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' (Cluster ' + str(cluster) + ')', parse_html=True)
    map_clusters.add_child(
        folium.features.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7))
       
map_clusters

# Cluster 0

In [35]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
135,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",0,Pharmacy,Intersection,Sushi Restaurant,Fish & Chips Shop,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Yoga Studio
315,Scarborough,"Milliken, Agincourt North, Steeles East, L'Amo...",0,Pharmacy,Intersection,Sushi Restaurant,Fish & Chips Shop,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Yoga Studio


# Cluster 1

In [36]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Not assigned,Not assigned,1,Coffee Shop,Clothing Store,Plaza,Restaurant,Fast Food Restaurant,Hotel,Seafood Restaurant,Cosmetics Shop,Women's Store,Breakfast Spot
1,Not assigned,Not assigned,1,Coffee Shop,Clothing Store,Plaza,Restaurant,Fast Food Restaurant,Hotel,Seafood Restaurant,Cosmetics Shop,Women's Store,Breakfast Spot
2,North York,Parkwoods,1,Spanish Restaurant,Fast Food Restaurant,Women's Store,Plaza,Supermarket,Restaurant,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market
3,North York,Victoria Village,1,Coffee Shop,Portuguese Restaurant,Hockey Arena,Intersection,French Restaurant,Pizza Place,Park,Fast Food Restaurant,Ethiopian Restaurant,Event Space
4,Downtown Toronto,"Regent Park, Harbourfront",1,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Food Truck,Spa,Dance Studio,Event Space,Beer Store,Electronics Store
...,...,...,...,...,...,...,...,...,...,...,...,...,...
355,Not assigned,Not assigned,1,Coffee Shop,Clothing Store,Plaza,Restaurant,Fast Food Restaurant,Hotel,Seafood Restaurant,Cosmetics Shop,Women's Store,Breakfast Spot
356,Not assigned,Not assigned,1,Coffee Shop,Clothing Store,Plaza,Restaurant,Fast Food Restaurant,Hotel,Seafood Restaurant,Cosmetics Shop,Women's Store,Breakfast Spot
357,Not assigned,Not assigned,1,Coffee Shop,Clothing Store,Plaza,Restaurant,Fast Food Restaurant,Hotel,Seafood Restaurant,Cosmetics Shop,Women's Store,Breakfast Spot
358,Etobicoke,"Mimico NW, The Queensway West, South of Bloor,...",1,Burrito Place,Yoga Studio,Gym / Fitness Center,Coffee Shop,Social Club,Middle Eastern Restaurant,Sushi Restaurant,Liquor Store,Thai Restaurant,Gym


# Cluster 2

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
77,North York,"North Park, Maple Leaf Park, Upwood Park",2,Bakery,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
99,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",2,Bakery,Asian Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Escape Room,Fish Market
257,North York,"North Park, Maple Leaf Park, Upwood Park",2,Bakery,French Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop
279,Scarborough,"Dorset Park, Wexford Heights, Scarborough Town...",2,Bakery,Asian Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Escape Room,Fish Market


# Cluster 3

In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,North York,Don Mills,3,Park,Gym,Pool,Trail,Gym / Fitness Center,River,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
20,North York,Don Mills,3,Park,Gym,Pool,Trail,Gym / Fitness Center,River,Escape Room,Ethiopian Restaurant,Event Space,Falafel Restaurant
23,York,Humewood-Cedarvale,3,Field,Park,Deli / Bodega,Playground,Hockey Arena,Trail,Grocery Store,Ethiopian Restaurant,Event Space,Falafel Restaurant
32,York,Caledonia-Fairbanks,3,Park,Gym,Bakery,Beer Store,Mexican Restaurant,Sporting Goods Shop,Women's Store,Food Court,Food & Drink Shop,Flower Shop
46,North York,Hillcrest Village,3,Residential Building (Apartment / Condo),Park,Field,Ethiopian Restaurant,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Yoga Studio,Electronics Store
54,Scarborough,Scarborough Village,3,Grocery Store,Spa,Park,Yoga Studio,Field,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop
55,North York,"Fairview, Henry Farm, Oriole",3,Park,Pharmacy,Thai Restaurant,Convenience Store,Coffee Shop,Wine Shop,Middle Eastern Restaurant,Pizza Place,Flower Shop,Flea Market
57,East York,"East Toronto, Broadview North (Old East York)",3,Intersection,Park,Convenience Store,Yoga Studio,Fish & Chips Shop,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field
64,North York,Bayview Village,3,Gas Station,Trail,Flower Shop,Park,Field,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Yoga Studio
93,Central Toronto,Lawrence Park,3,Photography Studio,Park,Yoga Studio,Fish & Chips Shop,Event Space,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Field,Fish Market


# Cluster 4

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1, 2] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Neighbourhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
108,Scarborough,"Wexford, Maryvale",4,Auto Garage,Yoga Studio,Ethiopian Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market
288,Scarborough,"Wexford, Maryvale",4,Auto Garage,Yoga Studio,Ethiopian Restaurant,Food Truck,Food Stand,Food Court,Food & Drink Shop,Flower Shop,Flea Market,Fish Market
